In [1]:
import scipy.io
import numpy as np
import utils
import matplotlib.pyplot as plt
import os
from numpy import linalg as LA
from scipy import signal
from scipy.linalg import toeplitz
from scipy.stats import zscore, pearsonr
from sklearn.covariance import LedoitWolf
from tqdm import tqdm

### Prepare data and pre-extracted features

- Source of data: EEGVolume.mat -- Samantha Cohen's EEG data with 64 electrodes from 18 subjects (https://www.parralab.org/isc/EEGVolume.mat) while watching this video (https://www.parralab.org/videos/roccos_AV.mp4). The length of the video is 3m 17s.
- EEG data is preprocessed and features of the video are extracted. Both with Dmochowski's code: https://github.com/dmochow/SRC
- The features can be optical flow, contrast, luminance, and even sound envelope since in their experiment the sound is not muted. ('muFlow', 'muSqFlow', 'muTemporalContrast', 'muSqTemporalContrast', 'muLuminance', 'muSqLuminance', 'muLocalContrast', 'muSqLocalContrast', 'stdLocalContrast', 'soundEnvelope', 'soundEnvelopeDown') The features are averaged over all pixels, giving us a scalar for each frame.
- Here we use optical flow because it gives a higher correlation (in this experiment). More specifically, we extract the average magnitude of the velocity in pixels per frame. Due to the Matlab CV toolbox version compatibility issue, the codes are rewritten a bit and the generated feature vector might be slightly different from the one generated by Dmochowski's original version.

In [2]:
# Load preprocessed EEG data
EEG_data = scipy.io.loadmat('../Correlated Component Analysis/data/Prepro_EEG.mat')
X_prepro = EEG_data['X']
fsEEG = 256 # fs of EEG data
T, D, N = X_prepro.shape # T=time/number of samples, D=number of channels, N=number of subjects 

In [3]:
# Load video features
features_data = scipy.io.loadmat('../Correlated Component Analysis/data/features.mat')
fsStim = int(features_data['fsVideo']) # fs of the video 
features = np.nan_to_num(features_data['muFlow']) # feature: optical flow

In [4]:
# Downsample EEG signals such that the sample frequency equals to the fs of the video
downsampledEEG = signal.resample_poly(X_prepro, fsStim, fsEEG)
features = features[:downsampledEEG.shape[0]]
normalized_features = zscore(features) # normalize features

###  Only use the data of one subject and perform CCA

In [5]:
# Select the EEG data of the first subject
EEG_of_one_sub = downsampledEEG[:,:,0]
# Length of the time filter that will be applied on the extracted features. 
# It equals to the fs of the video, meaning that we take the video signals from one second ago to the current moment into consideration.
L_timefilter = fsStim 

In [6]:
n_components = 5
# Find the convolution matrix and run CCA (with all data)
conv_mtx = utils.convolution_mtx(L_timefilter, normalized_features)
corr_coe, p_value, V_A, V_B = utils.cano_corr(EEG_of_one_sub, conv_mtx, n_components=n_components)
corr_coe

array([0.2989372 , 0.2285284 , 0.18205523, 0.17242502, 0.14464959])

#### Equivalence of CCA and GCCA when there are only two datasets
- A new function `GCCA_multi_modal` is added. It does not assume the input is pure EEG data of different subjects. The input can be a list of data of different modalities such as EEG data, video features, sound envelopes ... And it can also handle the case when the video feature is not a vector but a matrix, as long as the temporal/spatial/temporal-spatial filter is properly defined.
- Here we verify that with only two datasets (EEG data of one subject and stimulus features), the results of CCA and GCCA are the same.

In [7]:
datalist = [EEG_of_one_sub, conv_mtx]
_, W = utils.GCCA_multi_modal(datalist, n_components, regularization=None)
W_EEG = W[:D,:]
W_Stim = W[D:D+L_timefilter,:]

In [8]:
EEG_trans = EEG_of_one_sub@W_EEG
Stim_trans = conv_mtx@W_Stim
corr_pvalue = [pearsonr(EEG_trans[:,k], Stim_trans[:,k]) for k in range(n_components)]
corr_coe = np.array([corr_pvalue[k][0] for k in range(n_components)])
corr_coe

array([0.2989372 , 0.2285284 , 0.18205523, 0.17242502, 0.14464959])

#### Cross-Validation

10-fold cross-validation. Take the average of the results with different folds being test sets.

In [9]:
fold = 10
corr_train = np.zeros((fold, n_components))
corr_test = np.zeros((fold, n_components))
for idx in range(fold):
    EEG_train, EEG_test, Sti_train, Sti_test = utils.split(EEG_of_one_sub, normalized_features, fold=fold, fold_idx=idx+1)
    conv_mtx_train = utils.convolution_mtx(L_timefilter, Sti_train)
    corr_train[idx,:], p_value_train, V_A_train, V_B_train = utils.cano_corr(EEG_train, conv_mtx_train, n_components=n_components)
    conv_mtx_test = utils.convolution_mtx(L_timefilter, Sti_test)
    corr_test[idx,:], p_value_test, _, _ = utils.cano_corr(EEG_test, conv_mtx_test, n_components=n_components, V_A=V_A_train, V_B=V_B_train)

In [10]:
corr_train

array([[0.32771243, 0.24637156, 0.19444275, 0.18668884, 0.16651214],
       [0.3112457 , 0.23930184, 0.18907966, 0.17865862, 0.15109997],
       [0.30308641, 0.22480408, 0.18996318, 0.178692  , 0.15135686],
       [0.29794602, 0.26322925, 0.17445647, 0.15809979, 0.12367252],
       [0.32076544, 0.25866555, 0.19944495, 0.18510254, 0.14911758],
       [0.3084925 , 0.24042758, 0.1997938 , 0.17756234, 0.160543  ],
       [0.30212274, 0.24721223, 0.19427276, 0.17871493, 0.15366673],
       [0.30535755, 0.23594835, 0.19930048, 0.19085316, 0.16719047],
       [0.30534887, 0.24283177, 0.19402436, 0.18186287, 0.15363366],
       [0.30904631, 0.24244653, 0.1936564 , 0.18445021, 0.15229981]])

In [11]:
corr_test

array([[ 0.05079412,  0.04811422,  0.0349778 ,  0.01161869, -0.00677374],
       [ 0.00195257,  0.01284619,  0.01052971,  0.04390837, -0.01688414],
       [ 0.14638712,  0.19829973, -0.03253577,  0.00858125, -0.0392841 ],
       [ 0.08866005, -0.08008682, -0.01688248, -0.01830193, -0.02523443],
       [-0.00308839, -0.15099048, -0.03393569,  0.02582536,  0.06120831],
       [ 0.11201174,  0.02853912,  0.03749174,  0.05363409, -0.00507649],
       [ 0.22950709, -0.06514838,  0.00087503,  0.0723031 ,  0.01258612],
       [ 0.16819447,  0.02878553, -0.01181398, -0.03718242, -0.00887872],
       [ 0.16566291,  0.07835423, -0.03323929, -0.00630094,  0.00731673],
       [ 0.0404291 , -0.07510756, -0.02715303,  0.05357244,  0.04172316]])

The performance is quite dependent on which fold is the test set.

In [12]:
np.average(corr_train, axis=0)

array([0.3091124 , 0.24412388, 0.19284348, 0.18006853, 0.15290927])

In [13]:
np.average(corr_test, axis=0)

array([ 0.10005108,  0.00236058, -0.0071686 ,  0.0207658 ,  0.00207027])

### Make use of all data using GCCA

Feed all the EEG data and the convolution matrix of the features into GCCA. Returned weight vector can be decomposed as the weights of spatial filters of different subjects, and the weights of the time filter of extracted features.

In [14]:
datalist = [downsampledEEG, conv_mtx]
_, W = utils.GCCA_multi_modal(datalist, n_components, regularization='lwcov')
W_EEG = W[:N*D,:]
W_EEG_stack = np.reshape(W_EEG, (N,D,-1))
W_EEG_stack = np.transpose(W_EEG_stack, [1,0,2]) # W: D*N*n_components
W_Stim = W[N*D:N*D+L_timefilter,:]
Wlist = [W_EEG_stack, W_Stim]

In [15]:
avg_corr = utils.avg_corr_coe_multi_modal(datalist, Wlist, n_components=5)
avg_corr

array([0.1772279 , 0.15980386, 0.15726257, 0.15421245, 0.15289233])

#### Cross-Validation

In [16]:
fold = 10
cv_n_components = 5
corr_train = np.zeros((fold, cv_n_components))
corr_test = np.zeros_like(corr_train)
for i in range(fold):
    EEG_train, EEG_test, Sti_train, Sti_test = utils.split(downsampledEEG, normalized_features, fold=fold, fold_idx=i+1)
    _, W_train = utils.GCCA_multi_modal([EEG_train, Sti_train], cv_n_components, regularization='lwcov')
    W_EEG_train = W_train[:N*D,:]
    W_EEG_stack_train = np.reshape(W_EEG_train, (N,D,-1))
    W_EEG_stack_train = np.transpose(W_EEG_stack_train, [1,0,2]) # W: D*N*n_components
    W_Stim_train = W_train[N*D:N*D+L_timefilter,:]
    Wlist_train = [W_EEG_stack_train, W_Stim_train]
    corr_train[i,:] = utils.avg_corr_coe_multi_modal([EEG_train, Sti_train], Wlist_train, n_components=5)
    corr_test[i,:] = utils.avg_corr_coe_multi_modal([EEG_test, Sti_test], Wlist_train, n_components=5)


In [17]:
corr_train

array([[0.17539387, 0.16683442, 0.16101277, 0.1590582 , 0.15551872],
       [0.17487618, 0.17063546, 0.16013717, 0.15865898, 0.15373418],
       [0.172996  , 0.1643064 , 0.16045331, 0.15916425, 0.15555663],
       [0.17056082, 0.16462118, 0.1641981 , 0.15781858, 0.15341766],
       [0.17336186, 0.16464099, 0.16433894, 0.15934872, 0.15593266],
       [0.17064673, 0.1673626 , 0.1628272 , 0.16252107, 0.15245823],
       [0.168821  , 0.16665338, 0.16340172, 0.16009825, 0.15642094],
       [0.1687211 , 0.16642859, 0.16054201, 0.15758201, 0.15604066],
       [0.17009623, 0.16558072, 0.16212393, 0.16012822, 0.15617133],
       [0.17827071, 0.1691806 , 0.16616089, 0.16220081, 0.15537558]])

In [18]:
corr_test

array([[ 1.61146167e-02,  1.08403756e-02,  6.54611101e-03,
         1.81508247e-02,  1.11816757e-02],
       [ 1.76618215e-02, -3.18101813e-03,  7.81878981e-03,
         3.14057252e-03,  1.24491588e-02],
       [ 3.18872770e-02,  2.56613694e-02,  2.21237571e-02,
         3.07319575e-03,  1.98477390e-02],
       [ 3.90751916e-02,  4.86535164e-03,  3.54659331e-02,
         4.61215698e-03,  2.19893954e-02],
       [ 3.50060775e-02,  5.80221958e-03,  2.91052107e-02,
         1.15038484e-02,  1.40079087e-02],
       [ 4.16244509e-02,  1.69780682e-02,  9.44550660e-03,
        -4.18230572e-03,  8.26503850e-03],
       [ 5.87493742e-02,  1.53693317e-02,  2.52247307e-03,
         1.03479447e-02,  9.83778597e-03],
       [ 4.73821000e-02,  1.91308465e-02,  1.48959546e-03,
         3.27065447e-02,  1.28257155e-02],
       [ 3.48959394e-02,  2.79273262e-02,  1.29847449e-02,
         8.14563340e-03,  5.79975972e-04],
       [ 8.29320096e-05,  1.69663271e-03,  1.16593380e-03,
         2.58125516e-03

In [19]:
np.average(corr_train, axis=0)

array([0.17237445, 0.16662443, 0.1625196 , 0.15965791, 0.15506266])

In [20]:
np.average(corr_test, axis=0)

array([0.03224798, 0.01250905, 0.01286681, 0.00900797, 0.01145189])

- We can observe a large drop of correlation coefficients for unseen data, indicating the amount of data is insufficient. (The length of data is 3m 17s, and it is downsampled to 30 Hz.)
- The correlation of the first component of the last fold is very low (8.29320096e-05). It probably because the last 15 seconds of the video is all black. Then we are basically finding the correlation of the background EEG.

#### If we do not include features

- The result could be an indication of the quality of the features. Good features may guide the algorithm to do better EEG enhancement and thus lead to higher correlation. Bad features will drag the correlation down.
- Also if features are not included, then we can compare the results with the ones when EEG data is not downsampled and see to what extent the downsampling will affect the final result.

In [21]:
fold = 10
cv_n_components = 5
corr_train = np.zeros((fold, cv_n_components))
corr_test = np.zeros_like(corr_train)
T, D, N = downsampledEEG.shape
for i in range(fold):
    len_test = T // fold
    X_test = downsampledEEG[len_test*i:len_test*(i+1),:,:]
    X_train = np.delete(downsampledEEG, range(len_test*i, len_test*(i+1)), axis=0)
    _, W_train, _ = utils.GCCA(X_train, n_components=cv_n_components, regularization='lwcov')
    corr_train[i,:] = utils.avg_corr_coe(X_train, W_train, N, n_components=cv_n_components)
    corr_test[i,:] = utils.avg_corr_coe(X_test, W_train, N, n_components=cv_n_components)

In [22]:
np.average(corr_train, axis=0)

array([0.19235087, 0.18572353, 0.18103591, 0.17796426, 0.17295483])

In [23]:
np.average(corr_test, axis=0)

array([0.03680234, 0.01379785, 0.01454057, 0.01068417, 0.01319305])

- Slightly higher than including stimulus. 
- Quite different when EEG signals are not down sampled. 
    
    For reference, the results when EEG is not downsampled are:

    Training set: array([0.1828905 , 0.1031513 , 0.09170862, 0.09049872, 0.08981684])

    Test set: array([ 0.11516844,  0.02993057, -0.00094616,  0.00425634,  0.00174643])

#### Do not include features and compare GCCA with correlated component analysis

In [24]:
fold = 10
cv_n_components = 5
ISC_train = np.zeros((fold, cv_n_components))
ISC_test = np.zeros_like(ISC_train)
T, D, N = X_prepro.shape
for i in range(fold):
    len_test = T // fold
    X_test = X_prepro[len_test*i:len_test*(i+1),:,:]
    X_train = np.delete(X_prepro, range(len_test*i, len_test*(i+1)), axis=0)
    ISC_train[i,:], W_train = utils.corr_component(X_train, n_components=n_components)
    ISC_test[i,:], _ = utils.corr_component(X_test, n_components=n_components, W_train=W_train)

In [25]:
np.mean(ISC_train, axis=0)

array([0.05022358, 0.02988394, 0.02490273, 0.01870672, 0.01760152])

In [26]:
np.mean(ISC_test, axis=0)

array([ 0.04280363,  0.02148039,  0.01509772, -0.00111595,  0.00240869])

- The overfitting problem is suppressed compared to GCCA. And the performance on the test set is even slightly better.
- For reference, the results when EEG is not downsampled are:

    Training set: array([0.11514458, 0.04036184, 0.02881643, 0.02289647, 0.02139846])

    Test set: array([ 0.10402141,  0.02557607,  0.01402412,  0.00371617, -0.00083705])